# Zestaw zadań: Metoda najmniejszych kwadratów
## Zadanie 1
### Autor: Artur Gęsiarz

In [77]:
import pandas as pd
import matplotlib.pyplot as plt

### Importowanie danych do testowania

In [78]:
# pozyskiwanie nazw kolumn
column_names = []
with open("/Users/arturgesiarz/Desktop/MOwNiT/materialy/lab02/dataset/breast-cancer.labels") as f:
    for line in f:
        column_names.append(line.strip())

In [79]:
# pozyskianie pierwszego zestawu danych
df_cancer_train = pd.io.parsers.read_csv("/Users/arturgesiarz/Desktop/MOwNiT/materialy/lab02/dataset/breast-cancer-train.dat", names=column_names)

In [80]:
# pozyskianie pierwszego zestawu danych
df_cancer_validate = pd.io.parsers.read_csv("/Users/arturgesiarz/Desktop/MOwNiT/materialy/lab02/dataset/breast-cancer-validate.dat", names=column_names)
df_cancer_validate

,patient ID,Malignant/Benign,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),...,radius (worst),texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst)
0,892438,M,19.530,18.90,129.50,1217.0,0.11500,0.16420,0.219700,0.10620,...,25.93,26.24,171.10,2053.0,0.1495,0.41160,0.61210,0.19800,0.2968,0.09929
1,892604,B,12.460,19.89,80.43,471.3,0.08451,0.10140,0.068300,0.03099,...,13.46,23.07,88.13,551.3,0.1050,0.21580,0.19040,0.07625,0.2685,0.07764
2,89263202,M,20.090,23.86,134.70,1247.0,0.10800,0.18380,0.228300,0.12800,...,23.68,29.43,158.80,1696.0,0.1347,0.33910,0.49320,0.19230,0.3294,0.09469
3,892657,B,10.490,18.61,66.86,334.3,0.10680,0.06678,0.022970,0.01780,...,11.06,24.54,70.76,375.4,0.1413,0.10440,0.08423,0.06528,0.2213,0.07842
4,89296,B,11.460,18.16,73.59,403.1,0.08853,0.07694,0.033440,0.01502,...,12.68,21.61,82.69,489.8,0.1144,0.17890,0.12260,0.05509,0.2208,0.07638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,924934,B,10.290,27.61,65.67,321.4,0.09030,0.07658,0.059990,0.02738,...,10.84,34.91,69.57,357.6,0.1384,0.17100,0.20000,0.09127,0.2226,0.08283
256,924964,B,10.160,19.59,64.73,311.7,0.10030,0.07504,0.005025,0.01116,...,10.65,22.88,67.88,347.3,0.1265,0.12000,0.01005,0.02232,0.2262,0.06742
257,925236,B,9.423,27.88,59.26,271.3,0.08123,0.04971,0.000000,0.00000,...,10.49,34.24,66.50,330.6,0.1073,0.07158,0.00000,0.00000,0.2475,0.06969
258,925277,B,14.590,22.68,96.39,657.1,0.08473,0.13300,0.102900,0.03736,...,15.48,27.27,105.90,733.5,0.1026,0.31710,0.36620,0.11050,0.2258,0.08004


### Histogram i wykres wybranej kolumny